In [5]:
import pandas as pd
#from matplotlib import pyplot as plt
import numpy as np
import biosppy as bsp
import biosppy.signals.ecg as ecg
import neurokit as nk
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
import sklearn.externals.joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn import cross_validation, metrics
from sklearn.model_selection import GridSearchCV
from sklearn import svm
import time
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier


import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [6]:
dataset = pd.read_csv('/Users/Cristy/Downloads/task3/X_train.csv')
labels=pd.read_csv('/Users/Cristy/Downloads/task3/y_train.csv')
dataset['labels']=pd.Series(labels['y'])
dataset=dataset.drop(columns='id')
dataset.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x17804,x17805,x17806,x17807,x17808,x17809,x17810,x17811,x17812,labels
0,215,210,200,184,159,131,107,87,61,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,56,53,50,47,42,38,34,30,26,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,-65,-63,-61,-59,-57,-55,-52,-49,-46,-44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,38,67,100,131,161,191,219,249,267,275,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,-291,-279,-261,-224,-148,-64,11,66,71,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


### Features:

1. Distance between R peaks indices
1. Distance between T waves indices
1. Distance between P waves indices
1. Distance between Q waves indices
1. Distance between Q and P waves' amplitude
1. Distance between T and S waves' amplitude
1. Distance between S waves indices
1. R peak amplitude
1. P wave amplitude
1. Q wave amplitude
1. S wave amplitude
1. T wave amplitude
1. T wave duration
1. Time interval P wave-Q wave
1. Time interval S wave-T wave
1. RR Intervals
1. heart rate
1. heart rate diff
1. heart rate times diff


In [7]:
def mean_min_max_std(data):
    if len(data) == 0:
        return np.array([np.nan, np.nan, np.nan, np.nan])
    dmean = np.mean(data)
    dmin = np.min(data)
    dmax = np.max(data)
    dstd = np.std(data)
    return np.array([dmean, dmin, dmax, dstd])

N_FEATURES=76

def get_features(signal):
    features = np.empty(0)
    bio = nk.bio_process(ecg=signal, sampling_rate=sampling_rate)
    _, _, _, _, _, heart_rate_ts, heart_rate = ecg.ecg(signal=signal, sampling_rate=sampling_rate, show=False)
    
    # Add a bunch of features 
    features = np.append(features,mean_min_max_std(np.diff(bio['ECG']['R_Peaks'])))
    features = np.append(features,mean_min_max_std(np.diff(bio['ECG']['T_Waves'])))
    features = np.append(features,mean_min_max_std(np.diff(bio['ECG']['P_Waves'])))
    features = np.append(features,mean_min_max_std(np.diff(bio['ECG']['Q_Waves'])))
    features = np.append(features,\
                         mean_min_max_std([x-y for x,y in zip(bio['df'].ECG_Filtered.iloc[bio['ECG']['Q_Waves']],\
                                                              bio['df'].ECG_Filtered.iloc[bio['ECG']['P_Waves']])]))
    features = np.append(features,\
                         mean_min_max_std([x-y for x,y in zip(bio['df'].ECG_Filtered.iloc[bio['ECG']['T_Waves']],\
                                                              bio['df'].ECG_Filtered.iloc[bio['ECG']['S_Waves']])]))
    features = np.append(features,mean_min_max_std(np.diff(bio['ECG']['S_Waves'])))
    features = np.append(features,mean_min_max_std(bio['df'].ECG_Filtered.iloc[bio['ECG']['R_Peaks']]))
    features = np.append(features,mean_min_max_std(bio['df'].ECG_Filtered.iloc[bio['ECG']['P_Waves']]))
    features = np.append(features,mean_min_max_std(bio['df'].ECG_Filtered.iloc[bio['ECG']['Q_Waves']]))
    features = np.append(features,mean_min_max_std(bio['df'].ECG_Filtered.iloc[bio['ECG']['S_Waves']]))
    features = np.append(features,mean_min_max_std(bio['df'].ECG_Filtered.iloc[bio['ECG']['T_Waves']]))
    features = np.append(features,mean_min_max_std(np.diff(np.array(bio['ECG']['T_Waves_Ends'])-np.array(bio['ECG']['T_Waves_Onsets']))))
    features = np.append(features,mean_min_max_std(np.diff(np.array(bio['ECG']['Q_Waves'])-np.array(bio['ECG']['P_Waves']))))
    features = np.append(features,mean_min_max_std(np.diff(np.array(bio['ECG']['T_Waves'])-np.array(bio['ECG']['S_Waves']))))
    features = np.append(features,mean_min_max_std(bio['ECG']['HRV']['RR_Intervals']))
    features = np.append(features,mean_min_max_std(heart_rate))
    features = np.append(features,mean_min_max_std(np.diff(heart_rate)))
    features = np.append(features,mean_min_max_std(np.diff(heart_rate_ts)))
    
    return features


"""
Transform x ecg dataframe to features
"""
def transform_x(x):
    result = []
    for i in range(0, x.shape[0]):
        if i % 100 == 0:
            print(i)
        vals = x.iloc[i].values
        signal = vals[~np.isnan(vals)]
        try:
            features = get_features(signal)
        except:
            print('BROKEN {}'.format(i))
            features = np.full((N_FEATURES), np.nan) # a few rows break here, so just add nan row (it's only like 4 rows)
        result.append(features)
    return np.array(result)

In [9]:
x=np.array(dataset.sample(1).drop(columns='labels'))
sampling_rate=300
signal=x[~np.isnan(x)]
get_features(signal).shape

    Using multitaper spectrum estimation with 7 DPSS windows


(76,)

In [10]:
x_train_features = transform_x(dataset.drop(columns='labels'))

0
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using 

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hr

400
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Usin

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using mu

...

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows


    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using mu

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spe

In [11]:

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
x_data = imp.fit_transform(x_train_features)
y_train=labels.drop(columns='id')
y_train_values = y_train.values.ravel()
y_data = y_train_values

In [13]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,243.597015,98.0,256.0,22.185823,251.129032,218.0,504.0,33.234496,243.126984,125.0,...,124.185529,9.952609,0.011488,-37.602443,37.320107,7.029856,0.811568,0.326647,0.853283,0.074444
1,235.606061,208.0,257.0,12.761066,234.433333,164.0,274.0,20.196837,232.206897,154.0,...,85.586597,3.883948,-0.137922,-3.205128,2.411407,1.493474,0.784593,0.693250,0.856564,0.043021
2,269.551724,251.0,297.0,10.578398,271.153846,201.0,349.0,32.314193,271.040000,252.0,...,70.603353,2.230728,0.125242,-2.186039,2.006572,1.117426,0.898105,0.836565,0.989880,0.035847
3,123.376812,101.0,204.0,21.830773,857.250000,144.0,2813.0,834.007606,122.553846,97.0,...,170.026161,11.912734,-0.108934,-26.143791,25.886388,10.429553,0.409855,0.336628,0.679922,0.072426
4,179.697674,66.0,221.0,23.296406,205.200000,143.0,490.0,74.717296,179.923077,75.0,...,108.436358,6.004780,-0.052222,-10.608886,11.026965,3.222062,0.613908,0.503271,0.956549,0.072437


In [12]:
x_train=pd.DataFrame(x_data)
#x_train.to_csv("ECG_extracted_feats2.csv",index=False)

In [42]:
predictors = x_train.columns 
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=27)



Model Report
F1 Score :0.9656048465897987


TypeError: 'str' object is not callable

In [48]:
xgb_param=xgb1.get_xgb_params()
xgtrain=xgb.DMatrix(x_train[predictors].values, label=y_train.values.ravel())
cvresult=xgb.cv(xgb_param, xgtrain, num_boost_round=xgb1.get_params()['n_estimators'], nfold=5,
            metrics='mlogloss', early_stopping_rounds=50)

In [54]:
xgb_param

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 5,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 116,
 'nthread': 4,
 'objective': 'multi:softprob',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 27,
 'subsample': 0.8,
 'verbosity': 1,
 'num_class': 4}

In [38]:
kf=KFold(n_splits=10,shuffle=True,random_state=42)
folds=kf.split(x_data)

scores_train=[]
scores_test=[]
i=1

for train_idx, test_idx in folds:
    x_train_cv=x_data[train_idx]
    y_train_cv=y_data[train_idx]
    x_test_cv=x_data[test_idx]
    y_test_cv=y_data[test_idx]
    
    print('starts fold {}'.format(i))
    
    
    start_time = time.time()
    print('starts fitting')
    #abc.fit(x_train_cv,y_train_cv)
    xgb1.fit(x_train_cv,y_train_cv)
    print('starts prediction')
    train_preds=xgb1.predict(x_train_cv)
    end_time = time.time()
    total_time = round(end_time-start_time, ndigits=0)
    train_score=f1_score(y_train_cv,train_preds,average='micro')
    
    
    scores_train.append(train_score)
    
    test_preds=xgb1.predict(x_test_cv)
    test_score=f1_score(y_test_cv,test_preds,average='micro')
    scores_test.append(test_score)
    
    print('Fold:{}\n Train score: {}\n Test score: {}\n time ellapsed:{}'.format(i,train_score,test_score,total_time))
    i+=1
    

print('Average train score: {}\n Average test score: {}'.format(np.mean(scores_train),np.mean(scores_test)))
    

starts fold 1
starts fitting
starts prediction
Fold:1
 Train score: 0.9685124864277959
 Test score: 0.802734375
 time ellapsed:17.0
starts fold 2
starts fitting
starts prediction
Fold:2
 Train score: 0.9735070575461455
 Test score: 0.798828125
 time ellapsed:17.0
starts fold 3
starts fitting
starts prediction
Fold:3
 Train score: 0.9728555917480999
 Test score: 0.8046875
 time ellapsed:17.0
starts fold 4
starts fitting
starts prediction
Fold:4
 Train score: 0.971986970684039
 Test score: 0.794921875
 time ellapsed:18.0
starts fold 5
starts fitting
starts prediction
Fold:5
 Train score: 0.9715526601520087
 Test score: 0.765625
 time ellapsed:18.0
starts fold 6
starts fitting
starts prediction
Fold:6
 Train score: 0.970901194353963
 Test score: 0.81640625
 time ellapsed:18.0
starts fold 7
starts fitting
starts prediction
Fold:7
 Train score: 0.9724212812160695
 Test score: 0.81640625
 time ellapsed:18.0
starts fold 8
starts fitting
starts prediction
Fold:8
 Train score: 0.973295701259227

## Fix learning rate (0.1)

In [68]:
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=42)

params_grid={'n_estimators':[200,500,1000],'max_depth':[3,5,7,10]}
gscv=GridSearchCV(xgb1,params_grid,iid=False,cv=5,verbose=2,n_jobs=-1,scoring='f1_micro')
gscv.fit(x_train,y_train.values.ravel())


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 32.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=4, num_class=4,
                                     objective='multi:softprob', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=42, silent=None,
                                     subsample=0.8, verbosity=1),
             iid=False, n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 10],
                         'n_estimators': [200, 500, 1000]},
         

In [70]:
print(gscv.best_params_,gscv.best_score_)

{'max_depth': 3, 'n_estimators': 200} 0.8069118468413861


## Tune maximum depth and min_child_weight

In [71]:

xgb2 = xgb.XGBClassifier(
 n_estimators=200,
 max_depth=3,
 learning_rate =0.1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=42)

params2={'min_child_weight': range(1,6,2)}
gscv2=GridSearchCV(xgb2,params2,iid=False,cv=5,verbose=4,n_jobs=-1,scoring='f1_micro')

gscv2.fit(x_train,y_train.values.ravel())





Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  1.8min remaining:   26.4s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=200, n_jobs=1,
                                     nthread=4, num_class=4,
                                     objective='multi:softprob', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=42, silent=None,
                                     subsample=0.8, verbosity=1),
             iid=False, n_jobs=-1,
             param_grid={'min_child_weight': range(1, 6, 2)},
             pre_dispatch='2*n_jobs', refit=True, return_tra

In [72]:
print(gscv2.best_params_,gscv2.best_score_) #0.8069118468413861

{'min_child_weight': 1} 0.8069118468413861


## tune again number of estimators

In [74]:
xgb3 = xgb.XGBClassifier(
 max_depth=3,
 learning_rate =0.1,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=42)

params3={'n_estimators': [150,200,250]}
gscv3=GridSearchCV(xgb3,params3,iid=False,cv=5,verbose=4,n_jobs=-1,scoring='f1_micro')

gscv3.fit(x_train,y_train.values.ravel())




Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  1.7min remaining:   24.8s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=4, num_class=4,
                                     objective='multi:softprob', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=42, silent=None,
                                     subsample=0.8, verbosity=1),
             iid=False, n_jobs=-1, param_grid={'n_estimators': [150, 200, 250]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,


In [75]:
print(gscv3.best_params_,gscv3.best_score_) 

{'n_estimators': 200} 0.8069118468413861


## Tune gamma

In [78]:
params_gamm={'gamma':[i/10 for i in range (0,5)]}
xgb4 = xgb.XGBClassifier(
 n_estimators=200,
 max_depth=3,
 learning_rate =0.1,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=42)

gscv4=GridSearchCV(xgb4,params_gamm,iid=False,cv=5,verbose=4,n_jobs=-1,scoring='f1_micro')

gscv4.fit(x_train,y_train.values.ravel())




Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=0.8, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=200, n_jobs=1,
                                     nthread=4, num_class=4,
                                     objective='multi:softprob', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=42, silent=None,
                                     subsample=0.8, verbosity=1),
             iid=False, n_jobs=-1,
             param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
             pre_dispatch='2*n_jobs', refit=True, return_tra

In [79]:
print(gscv4.best_params_,gscv4.best_score_) 

{'gamma': 0.0} 0.8069118468413861


## Reduce learning rate

In [81]:
xgb5 = xgb.XGBClassifier(
 n_estimators=1000,
 max_depth=3,
 learning_rate =0.01,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=42)



kf=KFold(n_splits=10,shuffle=True,random_state=42)
folds=kf.split(x_data)

scores_test=[]
i=1

for train_idx, test_idx in folds:
    x_train_cv=x_data[train_idx]
    y_train_cv=y_data[train_idx]
    x_test_cv=x_data[test_idx]
    y_test_cv=y_data[test_idx]
    
    print('starts fold {}'.format(i))
    
    
    start_time = time.time()
    print('starts fitting')
    #abc.fit(x_train_cv,y_train_cv)
    xgb5.fit(x_train_cv,y_train_cv)
    print('starts prediction')

    end_time = time.time()
    total_time = round(end_time-start_time, ndigits=0)
   
    
   
    
    test_preds=xgb5.predict(x_test_cv)
    test_score=f1_score(y_test_cv,test_preds,average='micro')
    scores_test.append(test_score)
    
    print('Fold:{}\n Test score: {}\n time ellapsed:{}'.format(i,train_score,test_score,total_time))
    i+=1
    

print('Average train score: {}\n Average test score: {}'.format(np.mean(scores_train),np.mean(scores_test)))
    


starts fold 1
starts fitting
starts prediction
Fold:1
 Test score: 0.968519322622666
 time ellapsed:0.80859375
starts fold 2
starts fitting
starts prediction
Fold:2
 Test score: 0.968519322622666
 time ellapsed:0.802734375
starts fold 3
starts fitting
starts prediction
Fold:3
 Test score: 0.968519322622666
 time ellapsed:0.791015625
starts fold 4
starts fitting
starts prediction
Fold:4
 Test score: 0.968519322622666
 time ellapsed:0.791015625
starts fold 5
starts fitting
starts prediction
Fold:5
 Test score: 0.968519322622666
 time ellapsed:0.755859375
starts fold 6
starts fitting
starts prediction
Fold:6
 Test score: 0.968519322622666
 time ellapsed:0.810546875
starts fold 7
starts fitting
starts prediction
Fold:7
 Test score: 0.968519322622666
 time ellapsed:0.8046875
starts fold 8
starts fitting
starts prediction
Fold:8
 Test score: 0.968519322622666
 time ellapsed:0.796477495107632
starts fold 9
starts fitting
starts prediction
Fold:9
 Test score: 0.968519322622666
 time ellapsed:0

In [59]:
x_test = pd.read_csv('/Users/Cristy/Downloads/task3/X_test.csv').drop('id', axis=1)
x_test_features = transform_x(x_test)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
x_test_data = imp.fit_transform(x_test_features)

0
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7 DPSS windows
    Using

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using m

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
400
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper

NeuroKit Warning: ecg_process(): Sequence too short to compute heart rate.
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Sequence too short to compute interpolation. Will skip many features.
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using

...

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
2800
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitape

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using mu

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spe

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
3200
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitape

    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
NeuroKit Warning: ecg_hrv(): Not enough normal R peaks to compute HRV :/
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using mu

In [60]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

classifier=Pipeline(steps=[('smote',SMOTE(random_state=42)),('xgb1', xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=27))])



kf=KFold(n_splits=10,shuffle=True,random_state=42)
folds=kf.split(x_data)

scores_test=[]
i=1

for train_idx, test_idx in folds:
    x_train_cv=x_data[train_idx]
    y_train_cv=y_data[train_idx]
    x_test_cv=x_data[test_idx]
    y_test_cv=y_data[test_idx]
    
    print('starts fold {}'.format(i))
    
    
    start_time = time.time()
    print('starts fitting')
    #abc.fit(x_train_cv,y_train_cv)
    xgb1.fit(x_train_cv,y_train_cv)
    print('starts prediction')

    end_time = time.time()
    total_time = round(end_time-start_time, ndigits=0)
   
    
   
    
    test_preds=xgb1.predict(x_test_cv)
    test_score=f1_score(y_test_cv,test_preds,average='micro')
    scores_test.append(test_score)
    
    print('Fold:{}\n Test score: {}\n time ellapsed:{}'.format(i,train_score,test_score,total_time))
    i+=1
    

print('Average train score: {}\n Average test score: {}'.format(np.mean(scores_train),np.mean(scores_test)))
    


starts fold 1
starts fitting
starts prediction
Fold:1
 Test score: 0.968519322622666
 time ellapsed:0.8046875
starts fold 2
starts fitting
starts prediction
Fold:2
 Test score: 0.968519322622666
 time ellapsed:0.8046875
starts fold 3
starts fitting
starts prediction
Fold:3
 Test score: 0.968519322622666
 time ellapsed:0.80078125
starts fold 4
starts fitting
starts prediction
Fold:4
 Test score: 0.968519322622666
 time ellapsed:0.79296875
starts fold 5
starts fitting
starts prediction
Fold:5
 Test score: 0.968519322622666
 time ellapsed:0.751953125
starts fold 6
starts fitting
starts prediction
Fold:6
 Test score: 0.968519322622666
 time ellapsed:0.802734375
starts fold 7
starts fitting
starts prediction
Fold:7
 Test score: 0.968519322622666
 time ellapsed:0.796875
starts fold 8
starts fitting
starts prediction
Fold:8
 Test score: 0.968519322622666
 time ellapsed:0.7886497064579255
starts fold 9
starts fitting
starts prediction
Fold:9
 Test score: 0.968519322622666
 time ellapsed:0.8180

In [65]:
xgb1.fit(x_data,y_data)



y_test_pred = xgb1.predict(x_test_data)

file_test='xgbtest'
output = pd.DataFrame({'id':[float(i) for i in range(0,len(y_test_pred))],'y':y_test_pred})
output.to_csv("{}.csv".format(file_test),index=False)

In [67]:
classifier.fit(x_data,y_data)

y_test_pred = classifier.predict(x_test_data)

file_test='xgbSMOTEtest'
output = pd.DataFrame({'id':[float(i) for i in range(0,len(y_test_pred))],'y':y_test_pred})
output.to_csv("{}.csv".format(file_test),index=False)


In [83]:
from sklearn.ensemble import VotingClassifier
gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=3)

xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=42)

xgb5 = xgb.XGBClassifier(
 n_estimators=1000,
 max_depth=3,
 learning_rate =0.01,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softprob',
 num_class=4,
 nthread=4,
 scale_pos_weight=1,
 seed=42)



ensemble_clf = VotingClassifier(estimators=[
        ('gboost', gb), ('xgb1', xgb1), ('xgb5', xgb5)],
        voting='soft')

ensemble_clf.fit(x_data,y_data)

y_test_pred = classifier.predict(x_test_data)

file_test='ensemble3'
output = pd.DataFrame({'id':[float(i) for i in range(0,len(y_test_pred))],'y':y_test_pred})
output.to_csv("{}.csv".format(file_test),index=False)
